##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# On-device recommendation with TensorFlow Lite

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/lite/examples/recommendation/ml/ondevice_recommendation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/lite/examples/recommendation/ml/ondevice_recommendation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

This code base provides a toolkit to train and serve on-device recommendation
model. This approach personalizes recommendations by leveraging on-device data,
and protects user privacy without having user data leave device.

This Notebook shows an end-to-end example that 1) prepares sequential training data 2) trains neural-network model with various encoding techniques 3) exports the model to TensorFlow Lite 4) integrates in on-device ML applications to generate personalized recommendations.

With this example, we demonstrate the approach with public
[movielens](https://grouplens.org/datasets/movielens/) dataset, but you could
adapt the data processing script for your dataset and train your own
recommendation model.

## Model

We leverage a dual-encoder model architecture, with context-encoder to encode
sequential user history and label-encoder to encode predicted recommendation
candidate. Similarity between context and label encodings is used to represent
the likeliness predicted candidate meets user's needs.

Three different sequential user history encoding techniques are provided with
this code base:

* Bag of words encoder (BOW): averaging user activities' embeddings without
considering context order.
* Convolutional neural-network encoder (CNN): applying multiple layers of
convolutional neural-network to generate context encoding.
* Recurrent neural-network encoder (RNN): applying recurrent neural network
(LSTM in this example) to understand context sequence.


## Prerequisites

To run this example, please clone the source code from github [repo](https://github.com/tensorflow/examples/tree/master/lite/examples/recommendation/ml), install required packages.

In [ ]:
!git clone https://github.com/tensorflow/examples
!cd examples/lite/examples/recommendation/ml/
!pip install -r requirements.txt

## Training data preparation

This notebook makes use of public dataset [movielens](https://grouplens.org/datasets/movielens/) as training data for on-device movie recommendations. The data processing script performs the following steps:


*   Downloads [movielens](https://grouplens.org/datasets/movielens/) dataset
*   Groups movie rating records by user, and orders per-user movie rating records by timestamp.
*   Generates Tensorflow examples with features: 1) "context": time-ordered sequential movie IDs 2) "label": next movie ID user rated as label. "max_history_length" is taken in as parameter to define "context" feature shape, if not enough history found, right padding with out-of-vocab ID 0 will be performed.


Note: If you would like to use your own data, please adapt the data processing script for your specific case.


In [ ]:
!python -m data.example_generation_movielens \
  --data_dir=data/raw \
  --output_dir=data/examples \
  --build_movie_vocab=True \
  --min_timeline_length=3 \
  --max_context_length=10

Raw movielens ratings.dat data is in the following format:
UserID::MovieID::Rating::Timestamp

*   UserIDs range between 1 and 6040
*   MovieIDs range between 1 and 3952
*   Ratings are made on a 5-star scale (whole-star ratings only)
*   Timestamp is represented in seconds since the epoch as returned by time(2)
*   Each user has at least 20 ratings

Ref:[movielens readme.txt](http://files.grouplens.org/datasets/movielens/ml-1m-README.txt)

In this example, we consider each rating as a movie watch by the users, and construct user movie watch history with rated movie IDs ordering by time.

Sample generated training example with max user history as 10:
```
0 : {   # (tensorflow.Example)
  features: {   # (tensorflow.Features)
    feature: {
      key  : "context"
      value: {
        int64_list: {
          value: [ 595, 2687, 745, 588, 1, 2355, 2294, 783, 1566, 1907 ]
        }
      }
    }
    feature: {
      key  : "label"
      value: {
        int64_list: {
          value: [ 48 ]
        }
      }
    }
  }
}
```

# Train model

The training launcher script uses TensorFlow keras compile/fit APIs and performs
the following steps to kick start training and evaluation process:

*   Set up both train and eval dataset input function.
*   Construct keras model according to provided configs, please refer to sample.config file in the source code to config your model architecture, such as embedding dimension, convolutional neural network params, LSTM units etc.
*   Setup loss function. In this code base, we leverages customized batch softmax loss function.
*   Setup optimizer, with flag specified learning rate and gradient clip if needed.
*   Setup evaluation metrics, we provided recall@k metrics by default.
*   Compile model with loss function, optimizer and defined metrics.
*   Setup callbacks for tensorboard and checkpoint manager.
*   Run model.fit with compiled model, where you could specify number of epochs to train, number of train steps in each epoch and number of eval steps in each epoch.

To start training please execute command:


In [ ]:
!python -m model.recommendation_model_launcher_keras \
  --run_mode "train_and_eval" \
  --encoder_type "bow" \
  --training_data_filepattern "data/examples/train_movielens_1m.tfrecord" \
  --testing_data_filepattern "data/examples/test_movielens_1m.tfrecord" \
  --model_dir "model/model_dir" \
  --params_path "model/sample_config.json"\
  --batch_size 16 \
  --learning_rate 0.01 \
  --steps_per_epoch 1000 \
  --num_epochs 10000 \
  --num_eval_steps 1000 \
  --gradient_clip_norm 1.0 \
  --max_history_length 10

# Export model

Inside launcher script we also provide model exportation functionality.

In serve model, the model takes in user context history, for the example case the input is a vector of movie IDs you interacted with. With context encoder, model computes the context embedding vector, at the same time generate candidate embedding vector for all movie candidates in the vocab. By dotproduct and top-k ranking, top-k candidates will be served as the predicted candidates.

At model exportation step, you could specify number of predictions you want to get from the output of the model.

This step includes:


*   Export the model to saved_model with tf.saved_model.save.
*   Convert the saved_model to TensorFlow lite with tf.lite.TFLiteConverter.from_saved_model, and save it to the export directory wanted.

To export the model, please execute command:


In [ ]:
!python -m model.recommendation_model_launcher_keras \
  --run_mode "export" \
  --params_path "model/sample_config.json"\
  --model_dir "model/model_dir" \
  --checkpoint_path "model/model_dir/ckpt-1000" \
  --num_predictions 100

# Model inference

You could verify your model's performance by running inference with test examples.

In [ ]:
import tensorflow as tf

# Use [0, 1, ... 9] as example input to represent 10 movies that user interacted with.
context = tf.range(10)
# Directory to exported TensorFlow Lite model.
export_dir = ""
tflite_model_path = os.path.join(export_dir, 'model.tflite')
f = open(tflite_model_path, 'rb')
interpreter = tf.lite.Interpreter(model_content=f.read())
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

interpreter.set_tensor(input_details[0]['index'], context)
interpreter.invoke()
tflite_top_predictions_ids = interpreter.get_tensor(
    output_details[0]['index'])
tflite_top_prediction_scores = interpreter.get_tensor(
    output_details[1]['index'])

# Integrate in your application

We also open source an Android reference app to run inference with TF Lite.
**Please follow [`android/app/README.md`](https://github.com/tensorflow/examples/blob/master/lite/examples/recommendation/android/README.md)** to install required developer tools and build Android app.

The app uses one pretrained model to illustrate how to run TFLite. If you want to replace the existing model with the one you just trained above, please copy the respective TF Lite model to `assets` folder, and adapt its file name accordingly. If you directly train and export your model in this notebook, your
exported model should be located at "model/model_dir/export/model.tflite".

```shell
cp path/to/your/model.tflite ../android/app/src/main/assets/
```

The app uses the json file `config.json` to load one model and control how to consume IDs and scores predicted by the TF Lite recommendation model on device. `Config` definition can be found in [`android/app/src/main/java/org/tensorflow/lite/examples/recommendation/Config.java`](../android/app/src/main/java/org/tensorflow/lite/examples/recommendation/Config.java).

A sample json is presented below for the built-in model, and you may need to *adapt* it for your own trained model.

``` json
{
  "model": "model_history10_top100.tflite",
  "inputLength": 10,
  "outputLength": 100,
  "topK": 10,
  "movieList": "sorted_movie_vocab.json"
}
```
